In [ ]:
#this program will measure the "movement" seen in the camera
#it takes 2 images and finds the difference in pixel values to
#determine if some movement has occurred
#if there is some movement the Zumi will buzz

#note: dont move Zumi around as this
#will cause the image to change alot 
#and zumi will detect a lot of movement

from zumi.zumi import Zumi
from zumi.util.screen import Screen
from zumi.util.camera import Camera
import PIL.Image
import cv2
import IPython.display
import time



#if you set this to true the whole program slows down
#but it helps you understand what Zumi see's
show_image = True

#this is how sensitive you want Zumi to be. 
sensitivity = 8
#the higher the number the more sensitive
#lowest you should set it to is 1.5


#we will use the smallest resolution to speed things up
width = 80  #width pixel of image
height = 60 #height pixels of image 
time_out = 10 #this program should time out after 10 seconds

camera = Camera(width,height)
camera.start_camera()
zumi = Zumi()
screen = Screen()

#draw the eyes closed
screen.close_eyes()

#grab the time the program started
time_start = time.time()

#variable for the amount of time passed
time_elapsed = 0

#variable to keep track of the eyes/screen state
eyes_open = False
try:
    #program will continue to loop until the timeout ends
    while time_elapsed < time_out:
        
        #record the amount of time elapsed
        time_elapsed = time.time()-time_start
        
        #take 2 pictures
        image = camera.capture()
        #and turn them into gray images, no color
        gray1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        image = camera.capture()
        gray2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        #get both images and subtract them from each other
        frameDelta = cv2.absdiff(gray1, gray2)
        
        #set a threshold on the image where any pixel can only be either 1(black) or 0(white)
        #any pixel below min_pix_value will be black and anything above will be white 
        min_pix_value = 2
        ret, thresh = cv2.threshold(frameDelta,min_pix_value,255,cv2.THRESH_BINARY)
        
        #this finds all "blobs" of white that appear and finds their coordinates and sizes
        contoursArray = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)[-2]
        #check
        if len(contoursArray) > 0:
            #grab the biggest contour blob found
            countour = contoursArray[-1]# -1 goes the end of the list
            
            # find the x value, y value, width, and height of the rectangle
            # that bounds the contour
            x, y, w, h = cv2.boundingRect(countour)
            
            #the w is the width of the blob found and h the height
            #the area will be the product
            area = w*h
            
            print(area)
            
            #check if the area of movement is bigger than the threshold
            if(area > width*height/sensitivity):
                #if so there must be something moving 
                print("INTRUDER!")

                #make some sounds to notify the intruder you hate them.
                zumi.play_note(120,20)
                zumi.play_note(100,20)
                zumi.play_note(60,50)

                #you dont want to be drawing the screen everytime
                #so we need to make sure to use the screen ever time things change
                #drawing on the screens stops the program for 0.3 seconds
                if eyes_open == False:
                    #screen.hello()
                    screen.angry()
                    eyes_open = True
                    
            else:
                #turn of buzzer
                zumi.play_note(0,0)
                #if nothing is moving and the eyes are open, close them
                if eyes_open:
                    screen.close_eyes()
                    eyes_open = False
        if show_image == True:
            #this multiplier has to be in powers of 2 since they are pixels
            #pixels are discrete number you cant use 0.5 since there is no "half a pixel"
            multiplier = 4
            #resize the image so its a little bigger
            resized = cv2.resize(thresh, (width*multiplier, height*multiplier))
            #draw that image
            IPython.display.display(PIL.Image.fromarray(resized))
            
        IPython.display.clear_output(wait=True)
        
#you need this finally to stop the camera otherwise the red camera
#led will stay on and you wont be able to rerun the program.
finally:
    time.sleep(1)
    camera.close()
    print("done")    

None
